In [1]:
import sys

sys.path.insert(0, '..')

import shutil
import os
import uuid
from biosimulator_processes.execute import exec_utc_comparison
from verification_service.worker.compare import Supervisor
from verification_service.data_model.shared import MongoDbConnector
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os 

_outs = './test_outputs'
if os.path.exists(_outs):
    shutil.rmtree(_outs)


def jobid(): return str(uuid.uuid4())


load_dotenv("../verification_service/.env")
uri = os.getenv("MONGO_DB_URI")
omex_source_dir = './examples/sbml-core'
omex_name = 'Elowitz-Nature-2000-Repressilator.omex'
omex_fp = os.path.join(omex_source_dir, omex_name)
out_dir = './test_outputs'
simulators = ['amici', 'copasi', 'tellurium']
spec_name = 'cI mRNA'
job_id = jobid()
db_connector = MongoDbConnector(client=MongoClient(uri), database_id="service_requests")

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Cannot register SimpleNeuron. Error:
**
No module named 'pyneuroml'
**


In [2]:
supervisor = Supervisor(db_connector=db_connector)

In [3]:
supervisor.jobs

{'completed_jobs': ['ae6b1c6e-ecdc-421b-adce-2e977671d8ac',
  'a31b884b-b0ff-4170-a8ad-ee9a78d36bce'],
 'in_progress_jobs': ['47c8d515-679c-44c1-bfa4-28a9ccddcd55',
  'e981e546-4d07-4ea0-aaed-0efb9d0f5155'],
 'pending_jobs': ['e4033a27-f974-4521-8dcc-3c9367341ca8',
  'fd6e263a-9e6f-45a5-9d57-46ffee723fdd',
  'c2814ad8-71da-457f-880e-c47fcd018068']}

In [4]:
supervisor_response = supervisor.initialize()

Comparison already in progress and is probably also complete: test-comparison-0

Comparison already in progress and is probably also complete: test-comparison-1



In [5]:
supervisor_response

In [6]:
supervisor_response

In [7]:
supervisor.jobs

{'completed_jobs': ['ae6b1c6e-ecdc-421b-adce-2e977671d8ac',
  'a31b884b-b0ff-4170-a8ad-ee9a78d36bce'],
 'in_progress_jobs': ['47c8d515-679c-44c1-bfa4-28a9ccddcd55',
  'e981e546-4d07-4ea0-aaed-0efb9d0f5155'],
 'pending_jobs': ['e4033a27-f974-4521-8dcc-3c9367341ca8',
  'fd6e263a-9e6f-45a5-9d57-46ffee723fdd',
  'c2814ad8-71da-457f-880e-c47fcd018068']}

In [8]:
[v for v in supervisor.db_connector.db.completed_jobs.find()]

[{'_id': ObjectId('6677a286f2d3bfd0d5449107'),
  'job_id': 'ae6b1c6e-ecdc-421b-adce-2e977671d8ac',
  'status': 'COMPLETED',
  'timestamp': '2024-06-23 04:20:22.003486',
  'comparison_id': 'test-comparison-0',
  'results': None},
 {'_id': ObjectId('6677a286f2d3bfd0d5449109'),
  'job_id': 'a31b884b-b0ff-4170-a8ad-ee9a78d36bce',
  'status': 'COMPLETED',
  'timestamp': '2024-06-23 04:20:22.547622',
  'comparison_id': 'test-comparison-1',
  'results': None}]

In [9]:
job_id = supervisor.jobs['pending_jobs'][0]

In [10]:
job_id

'e4033a27-f974-4521-8dcc-3c9367341ca8'

In [11]:
job_doc = supervisor.db_connector.db.pending_jobs.find_one({'job_id': job_id})

In [12]:
job_doc

{'_id': ObjectId('6677a0d9654861cbb06f9657'),
 'status': 'PENDING',
 'job_id': 'e4033a27-f974-4521-8dcc-3c9367341ca8',
 'omex_path': './tmp/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'comparison_id': 'test-comparison-0',
 'timestamp': '2024-06-23 04:13:13.931620',
 'reports_path': './tmp/reports.h5',
 'include_output': True}

In [13]:
job_params = job_doc.copy()

In [14]:
list(map(lambda k: job_params.pop(k), ['job_id', 'status', 'timestamp', '_id']))

['e4033a27-f974-4521-8dcc-3c9367341ca8',
 'PENDING',
 '2024-06-23 04:13:13.931620',
 ObjectId('6677a0d9654861cbb06f9657')]

In [15]:
job_params

{'omex_path': './tmp/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'comparison_id': 'test-comparison-0',
 'reports_path': './tmp/reports.h5',
 'include_output': True}